<a href="https://colab.research.google.com/github/Hirenr12/PDAN-ICE-2/blob/main/Ice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import itertools

# 2. Download and Extract Dataset

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d puneet6060/intel-image-classification
!unzip intel-image-classification.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
intel-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  intel-image-classification.zip
replace seg_pred/seg_pred/10004.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
train_dir = "/content/seg_train/seg_train"
test_dir = "/content/seg_test/seg_test"

# 3. PREPROCESSING WITH ImageDataGenerator

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)


img_size = (150,150)
batch_size = 32


train_gen = train_datagen.flow_from_directory(
train_dir,
target_size=img_size,
batch_size=batch_size,
class_mode='categorical',
subset='training')


val_gen = train_datagen.flow_from_directory(
train_dir,
target_size=img_size,
batch_size=batch_size,
class_mode='categorical',
subset='validation')


test_gen = test_datagen.flow_from_directory(
test_dir,
target_size=img_size,
batch_size=batch_size,
class_mode='categorical',
shuffle=False)

Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


4. UNDERFITTING MODEL (Very Simple)

In [6]:
underfit_model = Sequential([
Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
MaxPooling2D(2,2),
Flatten(),
Dense(32, activation='relu'),
Dense(6, activation='softmax')
])


underfit_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


history_underfit = underfit_model.fit(train_gen, validation_data=val_gen, epochs=3)

Epoch 1/3
351/351 ━━━━━━━━━━━━━━━━━━━━ 150s 422ms/step - accuracy: 0.2923 - loss: 2.8020 - val_accuracy: 0.5756 - val_loss: 1.1034
Epoch 2/3
351/351 ━━━━━━━━━━━━━━━━━━━━ 196s 405ms/step - accuracy: 0.6512 - loss: 0.9171 - val_accuracy: 0.7008 - val_loss: 0.8395
Epoch 3/3
351/351 ━━━━━━━━━━━━━━━━━━━━ 204s 412ms/step - accuracy: 0.7784 - loss: 0.6355 - val_accuracy: 0.7090 - val_loss: 0.8858


# 5. OVERFITTING MODEL (Too Complex)

In [7]:
overfit_model = Sequential([
Conv2D(64, (3,3), activation='relu', input_shape=(150,150,3)),
Conv2D(64, (3,3), activation='relu'),
MaxPooling2D(2,2),
Conv2D(128, (3,3), activation='relu'),
Conv2D(128, (3,3), activation='relu'),
MaxPooling2D(2,2),
Conv2D(256, (3,3), activation='relu'),
Flatten(),
Dense(512, activation='relu'),
Dense(6, activation='softmax')
])


overfit_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


history_overfit = overfit_model.fit(train_gen, validation_data=val_gen, epochs=20)

Epoch 1/20
  1/351 ━━━━━━━━━━━━━━━━━━━━ 1:50:45 19s/step - accuracy: 0.1562 - loss: 1.7971

KeyboardInterrupt: 

# 6. FINAL MODEL (Balanced)

In [ ]:
final_model = Sequential([
Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
MaxPooling2D(2,2),
Conv2D(64, (3,3), activation='relu'),
MaxPooling2D(2,2),
Conv2D(128, (3,3), activation='relu'),
MaxPooling2D(2,2),
Flatten(),
Dense(128, activation='relu'),
Dropout(0.5),
Dense(6, activation='softmax')
])


final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


history_final = final_model.fit(train_gen, validation_data=val_gen, epochs=10, callbacks=[early_stop])

Epoch 1/10
 84/351 ━━━━━━━━━━━━━━━━━━━━ 7:17 2s/step - accuracy: 0.3559 - loss: 1.5579

# 7. EVALUATE FINAL MODEL

In [ ]:
loss, acc = final_model.evaluate(test_gen)
print(f"Final Test Accuracy: {acc:.2f}")


# Classification Report
y_true = test_gen.classes
y_pred_probs = final_model.predict(test_gen)
y_pred = np.argmax(y_pred_probs, axis=1)


print(classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys())))


# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_gen.class_indices.keys(), yticklabels=test_gen.class_indices.keys())
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# 8. TRAINING HISTORY PLOTS

In [ ]:
def plot_history(history, title):
plt.figure(figsize=(12,4))
# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title(f'{title} - Accuracy')
plt.legend()
# Loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title(f'{title} - Loss')
plt.legend()
plt.show()


plot_history(history_underfit, "Underfit Model")
plot_history(history_overfit, "Overfit Model")
plot_history(history_final, "Final Model")

# 9. MISCLASSIFIED EXAMPLES

In [ ]:
misclassified_idx = np.where(y_true != y_pred)[0]


plt.figure(figsize=(12,6))
for i, idx in enumerate(misclassified_idx[:5]):
img, label = test_gen[idx]
plt.subplot(1,5,i+1)
plt.imshow(img[0])
plt.title(f"True: {list(test_gen.class_indices.keys())[y_true[idx]]}\nPred: {list(test_gen.class_indices.keys())[y_pred[idx]]}")
plt.axis('off')
plt.show()

# 10. Let's Talk About the Models

Looking at how our models performed, we can see some interesting things:

**The Simple Model:** This first, very basic model just didn't cut it. It was too simple to really understand the images. Both its training and validation accuracy were pretty low, and the loss didn't drop as much as we'd hope. It's like trying to learn a complex subject with only a few basic ideas – you just underfit the problem.

**The Complex Model:** Then we went to the other extreme with a really complex model. This one got super good at recognizing the images it was trained on (high training accuracy!). But, when we showed it new images it hadn't seen before (the validation set), it didn't do as well, and its performance actually got worse over time. This is a classic case of overfitting – the model essentially memorized the training pictures instead of learning the general rules that apply to all pictures. It had too many parameters and got lost in the details.

**The Final Model:** This model seems to strike a better balance. It's more capable than the simple one, but we've added some techniques to keep it from overfitting like the complex one did.

*  Adding Dropout layers was a big help. Think of it like randomly making some of the model's connections take a break during training. This stops the model from relying too much on any one connection and forces it to find more general patterns, which helps it do better on new images.

* The convolutional layers are key to understanding the images, building up from simple edges to more complex shapes. We found that having a good number of these layers, but not too many, was important. Too few, and the model is too simple; too many, and it can get confused by noise and overfit.

*   **Finding the Right Training Time:** Training for too short a time means the model hasn't learned enough (underfitting). Training for too long means it starts memorizing instead of learning (overfitting). Using something called Early Stopping helped us here. It watched how the model was doing on the validation set and stopped training as soon as it started to get worse, making sure we kept the best version of the model.

*   **The Confusion Matrix Tells the Story:** While overall accuracy is nice, the confusion matrix gives us the real details. It shows us exactly where the model is getting confused – which types of images it's mixing up. This is super useful for figuring out what we might need to improve next.